In [214]:
import camelot.io as camelot
import pandas as pd
from datetime import datetime 
import requests
pd.set_option('display.max_rows', None)

In [215]:

from pathlib import Path

filename = Path('Proscribed-OrganizationsEng.pdf')
#url = 'https://nacta.gov.pk/wp-content/uploads/2017/08/Proscribed-OrganizationsEng.pdf'
#response = requests.get(url)
#filename.write_bytes(response.content)

In [216]:
tables = camelot.read_pdf('Proscribed-OrganizationsEng.pdf',flavor='lattice', table_areas=['27.890597738287564,779.3928781547891,586.183424878837,54.463041167308354'],line_size_scaling=5,split_text='true',flag_size='false')
#tables.export('pdfToTable.xls',f='excel')

In [217]:
tdf = tables[0].df

new_header = tdf.iloc[0] 

last_rows = tdf.tail(1)

tdf = tdf[1:] 

tdf.columns = new_header 
tdf.drop(columns=['S#'],inplace=True)

tdf2 = tdf.iloc[:,[2,3]]

#tdf2

tdf = tdf.iloc[:,[0,1]]


/home/data/anaconda3/envs/TestEnv/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [218]:

tdf_merged = pd.concat([tdf,tdf2])
tdf_merged = tdf_merged[tdf_merged['Name of Organization'] != '']

tdf_merged['Name of Organization'] = tdf_merged['Name of Organization'].str.replace('<[^<]+?>', '')
tdf_merged['Date of Notification'] = tdf_merged['Date of Notification'].str.replace('<[^<]+?>', '')

dumping_date = datetime.today().strftime('%Y-%m-%d')
#tdf.drop([tdf.columns[2] , tdf.columns[1]] ,  axis='columns', inplace=True)

In [219]:
tables_d = camelot.read_pdf('Proscribed-OrganizationsEng-1.pdf',flavor='stream', table_areas=['27.315145853073922,113.61750306703894,586.731731794465,55.07305706466556'],line_size_scaling=0,split_text='true',flag_size='false')

tdf_d = tables_d[0].df
tdf_d = tdf_d[tdf_d.iloc[:,[0]][0].str.contains('enlisted ') == False]
tdf_d2 = tdf_d.iloc[:,[2,3]]

tdf_d = tdf_d.iloc[:,[0,1]]

tdf_d.columns = tdf.columns
tdf_d2.columns = tdf.columns

tdf_merged2 = pd.concat([tdf_d,tdf_d2])


tdf_merged2 = tdf_merged2[tdf_merged2['Name of Organization'] != '']
tdf_merged2['Name of Organization'] = tdf_merged2['Name of Organization'].str.replace('<[^<]+?>', '')
tdf_merged2['Date of Notification'] = tdf_merged2['Date of Notification'].str.replace('<[^<]+?>', '')
tdf_merged2['Name of Organization'] = tdf_merged2['Name of Organization'].str.replace('\d+', '') 


tdf_merged = pd.concat([tdf_merged,tdf_merged2])
tdf_merged['Name of Organization'] = tdf_merged['Name of Organization'].str.replace('ffiliated', 'Affiliated') 
tdf_merged['data_source_name'] = 'nacta-entities'
tdf_merged['data_source_link'] = 'https://nacta.gov.pk/proscribed-organizations/'
tdf_merged['dumping_date'] = dumping_date
tdf_merged.rename(columns={'Name of Organization':'name','Date of Notification':'notification_date'}, inplace=True)
tdf_merged['notification_date'] = pd.to_datetime(tdf_merged['notification_date']).dt.strftime('%Y-%m-%d')



remove_words = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x','Organizations','organizations']
pat = r'\b(?:{})\b'.format('|'.join(remove_words))

new = tdf_merged["name"].str.split("Affiliated", n = 1, expand = True) 
  
# making separate first name column from new datmalba frame 
tdf_merged["name"]= new[0].str.strip()

tdf_merged["affiliated_entities"]= new[1].str.strip()
tdf_merged['affiliated_entities'] = tdf_merged['affiliated_entities'].str.replace(',', ' ||').str.replace('\n', '', 1).str.replace(pat, '').str.replace('[:.]', '').str.strip()
tdf_merged["affiliated_entities"]= tdf_merged["affiliated_entities"].str.split("\n",expand = False)

In [220]:
tdf_merged = tdf_merged.reset_index(drop=True)

dd = pd.DataFrame(tdf_merged[tdf_merged['affiliated_entities'].isna() == False]['affiliated_entities'].apply(lambda itemlist : [item for item in itemlist if item.isspace() == False]))
tdf_merged = tdf_merged.join(dd, lsuffix='_caller', rsuffix='_other')
tdf_merged.rename(columns = {'affiliated_entities_other':'affiliated_entities'}, inplace = True)
tdf_merged.drop(['affiliated_entities_caller'], axis = 1,inplace=True) 
tdf_merged

,name,notification_date,data_source_name,data_source_link,dumping_date,affiliated_entities
0,Lashkar-e-Jhangvi (LeJ),2001-08-14,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
1,Sipah-i-Muhammad Pakistan (SMP),2001-08-14,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
2,AJaish-e-Muhammad (JeM),2002-01-14,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,"[Al-Rehmat Trust || Bahawalpur , Al-Furqan Tr..."
3,Laskar-e-Tayyiba (LeT),2002-01-14,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
4,Sipah-i -Sahaba Pakistan (SSP),2002-01-14,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
5,Tehrik-e-Jaffria Pakistan (TJP),2002-01-28,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
6,Tehrik-e-Nifaz-e-Shariat-e-Mohammad (TNSM),2002-01-14,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
7,Tehreek-e-lslami,2002-01-14,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
8,Al-Qa'ida,2003-03-17,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN
9,Millat-e-lslamia Pakistan (Ex SSP),2003-11-15,nacta-entities,https://nacta.gov.pk/proscribed-organizations/,2020-04-17,NaN


In [221]:

#tdf_merged.to_csv('pdfToTable.csv',index=False)

tdf_merged.to_json(orient='records')

'[{"name":"Lashkar-e-Jhangvi (LeJ)","notification_date":"2001-08-14","data_source_name":"nacta-entities","data_source_link":"https:\\/\\/nacta.gov.pk\\/proscribed-organizations\\/","dumping_date":"2020-04-17","affiliated_entities":null},{"name":"Sipah-i-Muhammad Pakistan (SMP)","notification_date":"2001-08-14","data_source_name":"nacta-entities","data_source_link":"https:\\/\\/nacta.gov.pk\\/proscribed-organizations\\/","dumping_date":"2020-04-17","affiliated_entities":null},{"name":"AJaish-e-Muhammad (JeM)","notification_date":"2002-01-14","data_source_name":"nacta-entities","data_source_link":"https:\\/\\/nacta.gov.pk\\/proscribed-organizations\\/","dumping_date":"2020-04-17","affiliated_entities":["Al-Rehmat Trust || Bahawalpur "," Al-Furqan Trust || Karachi"]},{"name":"Laskar-e-Tayyiba (LeT)","notification_date":"2002-01-14","data_source_name":"nacta-entities","data_source_link":"https:\\/\\/nacta.gov.pk\\/proscribed-organizations\\/","dumping_date":"2020-04-17","affiliated_entitie